## Data Wrangling

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [5]:
#read in data
financials = pd.read_csv("/Users/sa12/Documents/Repositories/FinancialFraudDetection/FraudData/financials.csv", index_col=0).dropna()
#u
financials = financials[(financials['type'] == 'CASH_OUT') | (financials['type'] == 'TRANSFER')]

Using a discovery made in EDA, we can eliminate all types of transactions that are not CASH_OUT and TRANSFER. This is because fraud is only ever found in those two catagories. By eliminating a large chunk of data, we give more representation to our fraud data and help the model perform more accurately.

In [7]:
#drop unnecessary columns
financials = financials.drop(columns=['nameOrig', 'nameDest','isFlaggedFraud','type'])
#Rename existing columns to create unique, uniform identities
financials = financials.rename(columns={'amount':'Amount','oldbalanceOrg':'OldOrig','newbalanceOrig':'NewOrig','oldbalanceDest':'OldDest','newbalanceDest':'NewDest', 'isFraud':'Fraud'})
financials

,Amount,OldOrig,NewOrig,OldDest,NewDest,Fraud
step,,,,,,
1,181.00,181.00,0.0,0.00,0.00,1
1,181.00,181.00,0.0,21182.00,0.00,1
1,229133.94,15325.00,0.0,5083.00,51513.44,0
1,215310.30,705.00,0.0,22425.00,0.00,0
1,311685.89,10835.00,0.0,6267.00,2719172.89,0
...,...,...,...,...,...,...
743,339682.13,339682.13,0.0,0.00,339682.13,1
743,6311409.28,6311409.28,0.0,0.00,0.00,1
743,6311409.28,6311409.28,0.0,68488.84,6379898.11,1


In [8]:
from scipy import stats
#remove all rows that have outliers in at least one column
#identify all data with a Z score less than 3 and remove the outliers
#I have to remove each column except the fraud column. This is the most straightforward solution
clean_financials = financials
clean_financials = clean_financials[(np.abs(stats.zscore(clean_financials['Amount'])) < 3)]
clean_financials = clean_financials[(np.abs(stats.zscore(clean_financials['NewOrig'])) < 3)]
clean_financials = clean_financials[(np.abs(stats.zscore(clean_financials['OldOrig'])) < 3)]
clean_financials = clean_financials[(np.abs(stats.zscore(clean_financials['NewDest'])) < 3)]
clean_financials = clean_financials[(np.abs(stats.zscore(clean_financials['OldDest'])) < 3)]
clean_financials['Index'] = range(1,len(clean_financials)+1)
clean_financials = clean_financials.reset_index(drop=True).set_index('Index')
clean_financials.head()



We removed about 100,000 rows from the dataset that contained an outliers

In [9]:
numerical_columns = ['Amount','OldOrig','NewOrig','OldDest', 'NewDest']
# create a standard scaler object
scaler = StandardScaler()
# Scale the numerical columns
scaled_values = scaler.fit_transform(clean_financials[numerical_columns])
scaled_financials = pd.DataFrame(scaled_values, columns=(numerical_columns))
#put the isFraud column back to visualize fraud across distributions
scaled_financials = scaled_financials.merge(clean_financials['Fraud'], how='left', on=scaled_financials.index, right_index=False)
scaled_financials.drop(columns='key_0', inplace=True)
scaled_financials

,Amount,OldOrig,NewOrig,OldDest,NewDest,Fraud
0,-0.792964,-0.498552,-0.202408,-0.749266,-0.869591,1
1,-0.792964,-0.498552,-0.202408,-0.735043,-0.869591,1
2,-0.077207,-0.208755,-0.202408,-0.745853,-0.837262,0
3,-0.120422,-0.488524,-0.202408,-0.734209,-0.869591,0
4,0.180869,-0.294676,-0.202408,-0.745058,0.836943,0
...,...,...,...,...,...,...
2544929,0.156362,5.312424,-0.202408,-0.518516,-0.463220,1
2544930,0.014146,4.441898,-0.202408,-0.749266,-0.869591,1
2544931,0.014146,4.441898,-0.202408,-0.732361,-0.691649,1
2544932,-0.595274,0.711538,-0.202408,-0.749266,-0.869591,1


In [ ]:
#scaled_financials.to_csv('/Users/sa12/Documents/Repositories/FinancialFraudDetection/FraudData/financials.csv')